In [1]:
import pandas as pd
from collections import Counter
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv('./test.csv').drop(['id'], axis=1)
train

,date
0,1/1/2018 2:00
1,1/1/2018 5:00
2,1/1/2018 7:00
3,1/1/2018 8:00
4,1/1/2018 10:00
...,...
3499,31/12/2018 17:00
3500,31/12/2018 19:00
3501,31/12/2018 21:00
3502,31/12/2018 22:00


In [3]:
import time
def timeFormat(format_time):
    ts = time.strptime(format_time, "%d/%m/%Y %H:%M")
    return time.mktime(ts)

# print(timeFormat('1/1/2017 2:00'))

train['date'] = train['date'].map(timeFormat)
train

,date
0,1.514743e+09
1,1.514754e+09
2,1.514761e+09
3,1.514765e+09
4,1.514772e+09
...,...
3499,1.546247e+09
3500,1.546254e+09
3501,1.546261e+09
3502,1.546265e+09


In [4]:
time.localtime(1483207200.0)

time.struct_time(tm_year=2017, tm_mon=1, tm_mday=1, tm_hour=2, tm_min=0, tm_sec=0, tm_wday=6, tm_yday=1, tm_isdst=0)

In [5]:
train['year'] = train['date'].map(lambda x:time.localtime(x)[0])
train['month'] = train['date'].map(lambda x:time.localtime(x)[1])
train['day'] = train['date'].map(lambda x:time.localtime(x)[2])
train['hour'] = train['date'].map(lambda x:time.localtime(x)[3])
train['weekday'] = train['date'].map(lambda x:time.localtime(x)[6])
train

,date,year,month,day,hour,weekday
0,1.514743e+09,2018,1,1,2,0
1,1.514754e+09,2018,1,1,5,0
2,1.514761e+09,2018,1,1,7,0
3,1.514765e+09,2018,1,1,8,0
4,1.514772e+09,2018,1,1,10,0
...,...,...,...,...,...,...
3499,1.546247e+09,2018,12,31,17,0
3500,1.546254e+09,2018,12,31,19,0
3501,1.546261e+09,2018,12,31,21,0
3502,1.546265e+09,2018,12,31,22,0


In [6]:
weekend = [5, 6]

def is_weekend(x):
    if x in weekend:
        return 1
    else:
        return 0

In [7]:
train['is_weekend'] = train['weekday'].map(is_weekend)
train

,date,year,month,day,hour,weekday,is_weekend
0,1.514743e+09,2018,1,1,2,0,0
1,1.514754e+09,2018,1,1,5,0,0
2,1.514761e+09,2018,1,1,7,0,0
3,1.514765e+09,2018,1,1,8,0,0
4,1.514772e+09,2018,1,1,10,0,0
...,...,...,...,...,...,...,...
3499,1.546247e+09,2018,12,31,17,0,0
3500,1.546254e+09,2018,12,31,19,0,0
3501,1.546261e+09,2018,12,31,21,0,0
3502,1.546265e+09,2018,12,31,22,0,0


In [8]:
holiday_2017 = set([(1, 2), (1, 28), (1, 30), (1, 31), (4, 4), (4, 14), (4, 15), (4, 17), (5, 1), (5, 3), (5, 30),
               (7, 1), (10, 2), (10, 5), (10, 28), (12, 25), (12, 26)])
holiday_2018 = set([(1, 1), (2, 16), (2, 17), (2, 19), (3, 30), (3, 31), (4, 2), (4, 5), (5, 1), (5, 22), (6, 18),
                  (7, 2), (9, 25), (10, 1), (10, 17), (12, 25), (12, 26)])
# print(len(holiday_2018))
def is_holiday(x):
    year, month, day = time.localtime(x)[0], time.localtime(x)[1], time.localtime(x)[2]
    if year == 2017:
        if (month, day) in holiday_2017:
            return 1
        else:
            return 0
    if year == 2018:
        if (month, day) in holiday_2018:
            return 1
        else:
            return 0

In [9]:
train['is_holiday'] = train['date'].map(is_holiday)
train

,date,year,month,day,hour,weekday,is_weekend,is_holiday
0,1.514743e+09,2018,1,1,2,0,0,1
1,1.514754e+09,2018,1,1,5,0,0,1
2,1.514761e+09,2018,1,1,7,0,0,1
3,1.514765e+09,2018,1,1,8,0,0,1
4,1.514772e+09,2018,1,1,10,0,0,1
...,...,...,...,...,...,...,...,...
3499,1.546247e+09,2018,12,31,17,0,0,0
3500,1.546254e+09,2018,12,31,19,0,0,0
3501,1.546261e+09,2018,12,31,21,0,0,0
3502,1.546265e+09,2018,12,31,22,0,0,0


In [10]:
train['is_holiday'].sum()

167

In [11]:
train['free'] = train.apply(lambda train:train['is_weekend']+train['is_holiday'], axis=1)
train['free'] = train['free'].replace({2:1})
train

,date,year,month,day,hour,weekday,is_weekend,is_holiday,free
0,1.514743e+09,2018,1,1,2,0,0,1,1.0
1,1.514754e+09,2018,1,1,5,0,0,1,1.0
2,1.514761e+09,2018,1,1,7,0,0,1,1.0
3,1.514765e+09,2018,1,1,8,0,0,1,1.0
4,1.514772e+09,2018,1,1,10,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...
3499,1.546247e+09,2018,12,31,17,0,0,0,0.0
3500,1.546254e+09,2018,12,31,19,0,0,0,0.0
3501,1.546261e+09,2018,12,31,21,0,0,0,0.0
3502,1.546265e+09,2018,12,31,22,0,0,0,0.0


In [12]:
weather = pd.read_csv('./hongkong.csv').drop('location', axis=1)
# weather_2017['date'] = weather_2017['date'].map(lambda x:x[0:10])
weather = weather.rename(columns={'date_time':'date'})
weather

,date,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2017-01-01,21,17,0.0,8.7,5,20,09:03 AM,08:36 PM,07:03 AM,05:51 PM,15,19,20,19,23,30,80,0.0,1021,21,10,76,16
1,2017-01-02,22,17,0.0,8.7,5,27,09:47 AM,09:31 PM,07:03 AM,05:52 PM,17,21,21,21,14,11,80,0.0,1020,22,10,77,10
2,2017-01-03,22,18,0.0,8.7,5,34,10:29 AM,10:26 PM,07:04 AM,05:52 PM,17,21,22,21,18,8,81,0.0,1019,22,10,81,14
3,2017-01-04,22,18,0.0,8.7,5,41,11:11 AM,11:23 PM,07:04 AM,05:53 PM,17,21,22,21,20,8,82,0.0,1018,22,10,80,15
4,2017-01-05,22,18,0.0,5.5,5,49,11:53 AM,No moonset,07:04 AM,05:54 PM,18,21,21,21,14,42,83,0.0,1017,22,10,72,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,2018-12-27,22,20,0.0,8.7,6,62,10:44 PM,10:56 AM,07:01 AM,05:48 PM,15,22,23,22,15,43,66,0.1,1017,22,10,39,12
726,2018-12-28,20,15,0.0,8.7,5,55,11:45 PM,11:40 AM,07:01 AM,05:48 PM,10,18,18,18,23,27,61,0.0,1022,20,10,36,19
727,2018-12-29,14,12,0.0,8.7,4,48,No moonrise,12:21 PM,07:02 AM,05:49 PM,6,12,13,12,24,44,62,0.0,1026,14,10,21,20
728,2018-12-30,13,10,0.0,3.9,3,40,12:42 AM,01:01 PM,07:02 AM,05:50 PM,4,9,12,9,24,70,59,0.0,1027,13,10,10,20


In [13]:
for i in weather.columns:
    print(weather[i].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [14]:
weather.at[0, 'moonrise'][:2]

'09'

In [15]:
def map_am_pm(x):
    if x[-2:] == 'AM':
        return int(x[:2])
    elif x[-2:] == 'PM':
        return int(x[:2])+12
    else:
        return 0

to_change = ['moonrise', 'moonset', 'sunrise', 'sunset']
# weather['moonrise'] = weather['moonrise'].map(map_am_pm)
for i in to_change:
    weather[i] = weather[i].map(map_am_pm)

weather

,date,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2017-01-01,21,17,0.0,8.7,5,20,9,20,7,17,15,19,20,19,23,30,80,0.0,1021,21,10,76,16
1,2017-01-02,22,17,0.0,8.7,5,27,9,21,7,17,17,21,21,21,14,11,80,0.0,1020,22,10,77,10
2,2017-01-03,22,18,0.0,8.7,5,34,10,22,7,17,17,21,22,21,18,8,81,0.0,1019,22,10,81,14
3,2017-01-04,22,18,0.0,8.7,5,41,11,23,7,17,17,21,22,21,20,8,82,0.0,1018,22,10,80,15
4,2017-01-05,22,18,0.0,5.5,5,49,11,0,7,17,18,21,21,21,14,42,83,0.0,1017,22,10,72,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,2018-12-27,22,20,0.0,8.7,6,62,22,10,7,17,15,22,23,22,15,43,66,0.1,1017,22,10,39,12
726,2018-12-28,20,15,0.0,8.7,5,55,23,11,7,17,10,18,18,18,23,27,61,0.0,1022,20,10,36,19
727,2018-12-29,14,12,0.0,8.7,4,48,0,24,7,17,6,12,13,12,24,44,62,0.0,1026,14,10,21,20
728,2018-12-30,13,10,0.0,3.9,3,40,12,13,7,17,4,9,12,9,24,70,59,0.0,1027,13,10,10,20


In [16]:
# weather_2018 = pd.read_excel('./2018.xlsx', names=['date', 'high', 'low', 'weather', 'wind', 'strength'], header=None).drop('wind', axis=1)
# weather_2018['date'] = weather_2018['date'].map(lambda x:x[0:10])
# weather_2018

In [17]:
# def deal_weather(x):
#     if "大雨" in x or "暴雨" in x:
#         return 2
#     elif "雨" in x:
#         return 1
#     else:
#         return 0

In [18]:
# weather['weather'] = weather['weather'].map(deal_weather)
# weather

In [19]:
# wtemp = pd.get_dummies(weather['weather'], prefix='weather')
# weather = pd.concat([weather, wtemp], axis=1)

# stemp = pd.get_dummies(weather['strength'], prefix='strength')
# weather = pd.concat([weather, stemp], axis=1)

# weather

In [20]:
# weather.at[0, 'high'][:-1]

In [21]:
# weather['high'] = weather['high'].map(lambda x:int(x[:-1]))
# weather['low'] = weather['low'].map(lambda x:int(x[:-1]))

In [22]:
def timeFormat(format_time):
    ts = time.strptime(format_time, "%Y-%m-%d")
    return time.mktime(ts)

In [23]:
weather['date'] = weather['date'].map(timeFormat)
weather

,date,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,1.483200e+09,21,17,0.0,8.7,5,20,9,20,7,17,15,19,20,19,23,30,80,0.0,1021,21,10,76,16
1,1.483286e+09,22,17,0.0,8.7,5,27,9,21,7,17,17,21,21,21,14,11,80,0.0,1020,22,10,77,10
2,1.483373e+09,22,18,0.0,8.7,5,34,10,22,7,17,17,21,22,21,18,8,81,0.0,1019,22,10,81,14
3,1.483459e+09,22,18,0.0,8.7,5,41,11,23,7,17,17,21,22,21,20,8,82,0.0,1018,22,10,80,15
4,1.483546e+09,22,18,0.0,5.5,5,49,11,0,7,17,18,21,21,21,14,42,83,0.0,1017,22,10,72,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,1.545840e+09,22,20,0.0,8.7,6,62,22,10,7,17,15,22,23,22,15,43,66,0.1,1017,22,10,39,12
726,1.545926e+09,20,15,0.0,8.7,5,55,23,11,7,17,10,18,18,18,23,27,61,0.0,1022,20,10,36,19
727,1.546013e+09,14,12,0.0,8.7,4,48,0,24,7,17,6,12,13,12,24,44,62,0.0,1026,14,10,21,20
728,1.546099e+09,13,10,0.0,3.9,3,40,12,13,7,17,4,9,12,9,24,70,59,0.0,1027,13,10,10,20


In [24]:
# weather = weather.drop(['weather', 'strength'], axis=1)
# weather

# 14006 rows

In [25]:
train

,date,year,month,day,hour,weekday,is_weekend,is_holiday,free
0,1.514743e+09,2018,1,1,2,0,0,1,1.0
1,1.514754e+09,2018,1,1,5,0,0,1,1.0
2,1.514761e+09,2018,1,1,7,0,0,1,1.0
3,1.514765e+09,2018,1,1,8,0,0,1,1.0
4,1.514772e+09,2018,1,1,10,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...
3499,1.546247e+09,2018,12,31,17,0,0,0,0.0
3500,1.546254e+09,2018,12,31,19,0,0,0,0.0
3501,1.546261e+09,2018,12,31,21,0,0,0,0.0
3502,1.546265e+09,2018,12,31,22,0,0,0,0.0


In [26]:
weather

,date,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,1.483200e+09,21,17,0.0,8.7,5,20,9,20,7,17,15,19,20,19,23,30,80,0.0,1021,21,10,76,16
1,1.483286e+09,22,17,0.0,8.7,5,27,9,21,7,17,17,21,21,21,14,11,80,0.0,1020,22,10,77,10
2,1.483373e+09,22,18,0.0,8.7,5,34,10,22,7,17,17,21,22,21,18,8,81,0.0,1019,22,10,81,14
3,1.483459e+09,22,18,0.0,8.7,5,41,11,23,7,17,17,21,22,21,20,8,82,0.0,1018,22,10,80,15
4,1.483546e+09,22,18,0.0,5.5,5,49,11,0,7,17,18,21,21,21,14,42,83,0.0,1017,22,10,72,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,1.545840e+09,22,20,0.0,8.7,6,62,22,10,7,17,15,22,23,22,15,43,66,0.1,1017,22,10,39,12
726,1.545926e+09,20,15,0.0,8.7,5,55,23,11,7,17,10,18,18,18,23,27,61,0.0,1022,20,10,36,19
727,1.546013e+09,14,12,0.0,8.7,4,48,0,24,7,17,6,12,13,12,24,44,62,0.0,1026,14,10,21,20
728,1.546099e+09,13,10,0.0,3.9,3,40,12,13,7,17,4,9,12,9,24,70,59,0.0,1027,13,10,10,20


In [27]:
def timeFormat(format_time):
    ts = time.strptime(format_time, "%d/%m/%Y")
    return time.mktime(ts)

In [28]:
temp = pd.read_csv('./test.csv').drop(['id'], axis=1)
temp['date'] = temp['date'].map(lambda x:x.split()[0])
temp['date'] = temp['date'].map(timeFormat)
temp

,date
0,1.514736e+09
1,1.514736e+09
2,1.514736e+09
3,1.514736e+09
4,1.514736e+09
...,...
3499,1.546186e+09
3500,1.546186e+09
3501,1.546186e+09
3502,1.546186e+09


In [29]:
train['date'] = temp['date']
train

,date,year,month,day,hour,weekday,is_weekend,is_holiday,free
0,1.514736e+09,2018,1,1,2,0,0,1,1.0
1,1.514736e+09,2018,1,1,5,0,0,1,1.0
2,1.514736e+09,2018,1,1,7,0,0,1,1.0
3,1.514736e+09,2018,1,1,8,0,0,1,1.0
4,1.514736e+09,2018,1,1,10,0,0,1,1.0
...,...,...,...,...,...,...,...,...,...
3499,1.546186e+09,2018,12,31,17,0,0,0,0.0
3500,1.546186e+09,2018,12,31,19,0,0,0,0.0
3501,1.546186e+09,2018,12,31,21,0,0,0,0.0
3502,1.546186e+09,2018,12,31,22,0,0,0,0.0


In [30]:
# df.set_index('key').join(other.set_index('key'))
temp = train.set_index('date').join(weather.set_index('date'))
temp

,year,month,day,hour,weekday,is_weekend,is_holiday,free,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.514736e+09,2018,1,1,2,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
1.514736e+09,2018,1,1,5,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
1.514736e+09,2018,1,1,7,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
1.514736e+09,2018,1,1,8,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
1.514736e+09,2018,1,1,10,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.546186e+09,2018,12,31,17,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18
1.546186e+09,2018,12,31,19,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18
1.546186e+09,2018,12,31,21,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18


In [31]:
temp = temp.reset_index()
temp

,date,year,month,day,hour,weekday,is_weekend,is_holiday,free,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,1.514736e+09,2018,1,1,2,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
1,1.514736e+09,2018,1,1,5,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
2,1.514736e+09,2018,1,1,7,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
3,1.514736e+09,2018,1,1,8,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
4,1.514736e+09,2018,1,1,10,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,1.546186e+09,2018,12,31,17,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18
3500,1.546186e+09,2018,12,31,19,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18
3501,1.546186e+09,2018,12,31,21,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18
3502,1.546186e+09,2018,12,31,22,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18


In [32]:
temp = temp.drop(['date'], axis=1)
temp

,year,month,day,hour,weekday,is_weekend,is_holiday,free,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,sunset,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
0,2018,1,1,2,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
1,2018,1,1,5,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
2,2018,1,1,7,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
3,2018,1,1,8,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
4,2018,1,1,10,0,0,1,1.0,19,16,0.0,8.7,5,97,17,5,7,17,11,18,18,18,14,23,63,0.0,1020,19,10,65,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2018,12,31,17,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18
3500,2018,12,31,19,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18
3501,2018,12,31,21,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18
3502,2018,12,31,22,0,0,0,0.0,12,11,0.0,3.9,3,36,1,13,7,17,6,10,12,10,21,79,69,0.0,1028,12,10,138,18


In [33]:
temp.to_csv('./test_features.csv', encoding='utf-8', index=False)

In [33]:
# features.to_csv('./features.csv', encoding='utf-8', index=False)
# labels.to_csv('./labels.csv', encoding='utf-8', index=False)